In [1]:
import pandas as pd
# surprise libraray for recommendation system
from surprise import SVD
from surprise import dataset
from surprise import Reader
from surprise import model_selection
#from surprise import evaluate, print_perf

In [2]:
# A small test
# DF creation. Replace this by your own df
#ratings_dict = {'userID': [9, 32, 2, 45, 1],
#                'itemID': [1, 1, 1, 2, 2],
#                'rating': [3, 2, 4, 3, 1]}
#df = pd.DataFrame.from_dict(ratings_dict)

# You'll need to create a dummy reader
#reader = Reader(line_format='user item rating', rating_scale=(1, 5))

# Also, a dummy Dataset class for surprise
#class MyDataset(dataset.DatasetAutoFolds):
#    def __init__(self, df, reader):
#        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in zip(df['userID'], df['itemID'], df['rating'])]
#        self.reader=reader

#data = MyDataset(df, reader)

# You should now be able to use the data object normally.
#algo = SVD()
#model_selection.validation.cross_validate(algo, data, measures=[u'rmse', u'mae'], cv=3, return_train_measures=False, n_jobs=-1, pre_dispatch=u'2*n_jobs', verbose=True)

In [3]:
# construct dataframe for train...
df_train = pd.read_csv('../../data/dataset/csv/review.csv', index_col = 'review_id')
import sklearn.utils
sub_df_train = sklearn.utils.shuffle(df_train).iloc[:100000, :]
sub_df_train.head()

,useful,date,text,stars,business_id,user_id,cool,funny
review_id,,,,,,,,
Jod7ChQTONJ-h45N_69ZUg,0,2017-08-25,Extremely expensive steakhouse where you are m...,3,L_ZLtfHvfzfoNVQ0-okTXg,Gs9MUwjkKGGJeOv17dfEnA,0,0
NUqCU1VMcojpHdgafIauXg,1,2017-01-03,We were in town for work and decided to take s...,4,GHS1rVjO-RMcRB6WJLpCDQ,-lLEwwXKARlLhAd05tfmVA,1,0
YK-_nspTyhnpyjtNYVd2EQ,1,2011-07-22,Waited about 20 minutes before I was able to o...,5,RwMLuOkImBIqqYj4SSKSPg,EUAtnptDql_tMRsETuELkg,0,0
F2sSwj0sviCBlmekcA7nrg,0,2014-07-26,I came here for the hakka food. The waiter was...,2,WmuoWetYC-4U2-V4taC6Wg,-d2daWmftYumOaYpbD5D8Q,0,0
6KUBC6tZIPdvx7A6jjs3-Q,0,2014-08-23,Excellent Korean food by Arizona standards. Si...,5,UO_62NNq1pKY61teaoN2bQ,sk9mVBac5xcQY2-H-9X3hQ,0,0


In [4]:
def CVScoreWithSurprise(df, nfold=3, algo=SVD()):
    reader = Reader(line_format='user item rating', rating_scale=(1, 5))
    # A dummy Dataset class for surprise
    class MyDataset(dataset.DatasetAutoFolds):
        def __init__(self, df, reader):
            self.raw_ratings = [(uid, bid, r, None) for (uid, bid, r) in zip(df['user_id'], df['business_id'], df['stars'])]
            self.reader=reader
    # Prepare for the surprise dataset, from pandas df
    data = MyDataset(df, reader)
    model_selection.validation.cross_validate(algo, data, measures=[u'rmse', u'mae'], cv=nfold, return_train_measures=False, n_jobs=-1, pre_dispatch=u'2*n_jobs', verbose=True)

In [5]:
# imprort more algorithms...
# basic
from surprise import NormalPredictor
from surprise import BaselineOnly
# k-NN
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
# matrix factorization
from surprise import SVDpp
from surprise import NMF

algo_dict = {
    'Random': NormalPredictor(),
    'Baseline': BaselineOnly(),
    #'k-NN, Basic': KNNBasic(),
    #'k-NN, Means': KNNWithMeans(),
    #'k-NN, ZScore': KNNWithZScore(),
    #'k-NN, Baseline': KNNBaseline(),
    'SVD': SVD(),
    'SVD++': SVDpp(),
    'NMF': NMF(),
}
for key, value in algo_dict.items():
    print("Cross validation result for %s algorithm: "%key)
    CVScoreWithSurprise(sub_df_train, nfold=5, algo=value)
    print("Done!")
    print("\n")

Cross validation result for Random algorithm: 
Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8515  1.8541  1.8564  1.8576  1.8647  1.8569  0.0045  
MAE (testset)     1.4671  1.4739  1.4771  1.4777  1.4821  1.4755  0.0050  
Fit time          0.17    0.17    0.19    0.17    0.18    0.18    0.01    
Test time         0.14    0.14    0.14    0.15    0.14    0.14    0.00    
Done!


Cross validation result for Baseline algorithm: 
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4085  1.4035  1.4005  1.3956  1.3948  1.4006  0.0051  
MAE (testset)     1.1826  1.1763  1.1731  1.1701  1.1725  1.1749  0.0043  
Fit time    